In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import requests

def ask_llm(text:str):
    url = "http://localhost:5001/chat"

    data = [
        {"id": 1, "text": text},
    ]

    response = requests.post(url, json=data)

    return response .json()[0]["response"]


text = "Qual è la capitale della Francia?"

answer = ask_llm(text)

#("Status Code:", response.status_code)
# print("Response JSON:", response.json())


In [ ]:
def say(text: str, language: str = "en"):
    url = "http://localhost:5001/tts"

    data = [
        {"id": 1, "text": text, "language": language},
    ]

    response = requests.post(url, json=data)
    result = response.json()[0]

    if "status" in result and result["status"] == "success":
        print(f"[TTS] Audio saved to: {result['file']}")
    else:
        print(f"[TTS] Error: {result.get('error', 'Unknown error')}")


say(answer, language="it")

In [ ]:
# import pyaudio
# import wave
# import os

# def hear(audio_output_path: str, device_index: int = None):
#     """
#     @param audio_output_path: where to save the mic input got by this func.
#     @param device_index: index of the input device (see list below)
#     """


#     # Recording parameters
#     CHUNK = 2048
#     FORMAT = pyaudio.paInt32  # 32-bit integer format for S32_LE
#     CHANNELS = 1              # 2 for stereo, 1 for mono
#     RATE = 48000              # 48000 Hz or lower?
#     RECORD_SECONDS = 5

#     # List devices if device_index is not provided
#     p = pyaudio.PyAudio()
#     if device_index is None:
#         print("Available audio input devices:")
#         for i in range(p.get_device_count()):
#             info = p.get_device_info_by_index(i)
#             if info['maxInputChannels'] > 0:
#                 print(f"  Index {i}: {info['name']}")
#         device_index = int(input("Enter the device index for your digital mic: "))

#     # Open stream
#     stream = p.open(format=FORMAT,
#                     channels=CHANNELS,
#                     rate=RATE,
#                     input=True,
#                     input_device_index=device_index,
#                     frames_per_buffer=CHUNK)

#     print("Recording...")
#     frames = []

#     for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
#         data = stream.read(CHUNK, exception_on_overflow=False)      # continue recording when exceptions occur
#         frames.append(data)

#     print("Recording finished.")

#     stream.stop_stream()
#     stream.close()
#     p.terminate()

    

#     # Save as WAV file
#     os.makedirs(os.path.dirname(audio_output_path), exist_ok=True)
#     wf = wave.open(audio_output_path, 'wb')
#     wf.setnchannels(CHANNELS)
#     wf.setsampwidth(p.get_sample_size(FORMAT))
#     wf.setframerate(RATE)
#     wf.writeframes(b''.join(frames))
#     wf.close()

#     print(f"Audio saved as {audio_output_path}")
#     return audio_output_path


# def stt(audio_output_path):
#     pass


# audio_file_path = hear("src/test.wav", 4)
# stt(audio_file_path)

In [7]:
import requests
import pyaudio
import wave
import io

def hear(duration:int = 5, device_index: int = None):
    """Record audio and return it as bytes in WAV format"""
    
    # Recording parameters
    CHUNK = 2048
    FORMAT = pyaudio.paInt32
    CHANNELS = 1
    RATE = 48000
    RECORD_SECONDS = duration
    
    p = pyaudio.PyAudio()
    
    # List devices if needed
    if device_index is None:
        print("Available audio input devices:")
        for i in range(p.get_device_count()):
            info = p.get_device_info_by_index(i)
            if info['maxInputChannels'] > 0:
                print(f"  Index {i}: {info['name']}")
        device_index = int(input("Enter the device index for your digital mic: "))
    
    # Open stream
    stream = p.open(format=FORMAT,
                    channels=CHANNELS,
                    rate=RATE,
                    input=True,
                    input_device_index=device_index,
                    frames_per_buffer=CHUNK)
    
    print("Recording...")
    frames = []
    
    for _ in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK, exception_on_overflow=False)
        frames.append(data)
    
    print("Recording finished.")
    
    stream.stop_stream()
    stream.close()
    
    # Create WAV file in memory
    wav_buffer = io.BytesIO()
    wf = wave.open(wav_buffer, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(FORMAT))
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    p.terminate()
    
    # Get the WAV data as bytes
    wav_buffer.seek(0)
    return wav_buffer.read()


def transcribe(audio_data):
    # Send directly to your STT endpoint
    url = "http://localhost:5001/stt"
    files = {"file": ("audio.wav", audio_data, "audio/wav")}
    response = requests.post(url, files=files)
    return response.json()

# Record audio
audio_data = hear(duration=3, device_index=4)
# transcription
print("Transcription:", transcribe(audio_data))



ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.front
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround21
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround40
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround41
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround50
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround51
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.surround71
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2722:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.iec958
ALSA lib pcm.c:2722:(snd

Recording...
Recording finished.
Transcription: {'text': ' Fallo, ti prego fallo Vieni qua Antonio vieni'}
